# Simulando uma ligação para o Call Center da Quantum Finance, um projeto de FinTech da FIAP

## Detalhamento dos requisitos

Gerar as frases para simular um atendimento telefônico, que deverá *identificar* a empresa, *saudar* o cliente e *oferecer* as opções a seguir: 
- (1) Consulta ao saldo da conta;
- (2) Simulação de compra internacional;
- (3) Falar com um atendente;
- (4) Sair do atendimento.

As frases devem ser geradas usando alguma das bibliotecas de TTS disponíveis, e armazenadas em arquivo para serem usadas dentro do loop. 
Gerar também as frases de resposta correspondentes para confirmar a identificação
de cada opção. 
Por exemplo: “Você escolheu ‘’Falar com um atendente’’.”

In [1]:
import io
import speech_recognition as SpeechRec
import unicodedata
import simpleaudio as sa

from pydub import AudioSegment
from gtts import gTTS
from IPython.display import Audio

In [2]:
script_texts = {
    "saudacao": "Olá, você ligou para Quantum Finance, onde quantu mais você investe, melhor!",
    "opcoes": "Escolha entre as seguinte opções:\n\nOpção 1, Consulta ao saldo da conta;\nOpção 2, Simulação de compra internacional;\nOpção 3, Falar com um atendente;\nOpção 4, Sair do atendimento.\n",
    "opcao1_resposta": "Você escolheu: Opção 1, Consulta ao saldo da conta;",
    "opcao2_resposta": "Você escolheu: Opção 2, Simulação de compra internacional;",
    "opcao3_resposta": "Você escolheu: Opção 3, Falar com um atendente;",
    "opcao4_resposta": "Você escolheu: Opção 4, Sair do atendimento.",
    "nao_encontrada": "Não entendi o que falou, vou repetir as opções:",
}

In [3]:
saveOn = './files'

for title in script_texts:
    newAudio = gTTS(script_texts[title], lang='pt')
    newAudio.save(f"{saveOn}/stage/{title}.mp3")

    # Convert to WAV
    audio = AudioSegment.from_mp3(f"{saveOn}/stage/{title}.mp3")
    audio.export(f"{saveOn}/{title}.wav", format="wav")

Construir um loop que, uma vez iniciado, irá executar os áudios iniciais com as opções. Dentro do loop, o algoritmo
deverá capturar o áudio do usuário usando alguma das **bibliotecas de STT** e tentar identificar a opção selecionada.
Não é necessário usar nada avançado, uma busca por palavra-chave simples é suficiente.
Uma vez que a opção seja identificada, o áudio de resposta correspondente deverá ser tocado.

In [4]:
SpeechRec.Microphone.list_working_microphones()

for index, name in enumerate(SpeechRec.Microphone.list_microphone_names()):
    print("Microphone with name \"{1}\" found for `Microphone(device_index={0})`".format(index, name))

recognizer = SpeechRec.Recognizer()

Microphone with name "NDI Audio" found for `Microphone(device_index=0)`
Microphone with name "External Microphone" found for `Microphone(device_index=1)`
Microphone with name "External Headphones" found for `Microphone(device_index=2)`
Microphone with name "MacBook Air Microphone" found for `Microphone(device_index=3)`
Microphone with name "MacBook Air Speakers" found for `Microphone(device_index=4)`
Microphone with name "Microsoft Teams Audio" found for `Microphone(device_index=5)`
Microphone with name "NDI Audio" found for `Microphone(device_index=6)`


In [5]:
def normalize(texto):
    texto = unicodedata.normalize('NFKD', texto.lower())
    return ''.join(c for c in texto if not unicodedata.combining(c))

def play_audio(file_path):
    wave_obj = sa.WaveObject.from_wave_file(file_path)
    play_obj = wave_obj.play()
    play_obj.wait_done()

def not_founded():
    audio_file_path = f'{saveOn}/nao_encontrada.wav'
    play_audio(audio_file_path)

def select_option(option):
    audio_file_path = f'{saveOn}/opcao{option}_resposta.wav'
    play_audio(audio_file_path)

    print(option)

In [7]:
input_selected = 1 # Altere aqui para seu microfone
in_call = True

audio_file_path = f'{saveOn}/saudacao.wav'
play_audio(audio_file_path)

while in_call:
    audio_file_path = f'{saveOn}/opcoes.wav'
    play_audio(audio_file_path)
    
    with SpeechRec.Microphone(input_selected) as inputDevice:
        recognizer.adjust_for_ambient_noise(inputDevice,duration=1)
        print('Listening...')
        audio = recognizer.listen(inputDevice)

    try:
        text = recognizer.recognize_google(audio, language='pt-BR')
        normalized_text = normalize(text)
        print(normalized_text)

        if normalized_text in normalize(script_texts['opcao1_resposta']):
            select_option(1)
            in_call = False
        elif normalized_text in normalize(script_texts['opcao2_resposta']):
            select_option(2)
            in_call = False
        elif normalized_text in normalize(script_texts['opcao3_resposta']):
            select_option(3)
            in_call = False
        elif normalized_text in normalize(script_texts['opcao4_resposta']):
            select_option(4)
            in_call = False
        else:
            not_founded()
    except SpeechRec.UnknownValueError:
        print("Não consegui entender o áudio")
        not_founded()
    except SpeechRec.RequestError as e:
        print(f"Erro ao solicitar resultados do serviço Google Speech Recognition; {e}")
        not_founded()

Listening...
japao
Listening...
opcao 4
4
